In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dat
import matplotlib as mplt
import datetime as dt
import xgboost as xgb
import functools as functools
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline
sns.set()

In [2]:
installs = pd.read_csv("data/installs.csv", dtype={'auction_type_id': 'float64', 'country': 'int64', 'date': 'str',
                                                   'device_id': 'int64', 'platform':'int64', 'ref_type_id': 'int64',
                                                   'source_id':'int64','ref_hash':'category' }, parse_dates = ['created'],low_memory=False)

installs.head(5)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,6287817205707153877,NaN,3.739127e+17,adjust.com,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.301378e+18
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,6287817205707153877,NaN,7.805539e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.301378e+18
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060930e+18
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.301378e+18
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.301378e+18


In [3]:
installs=installs.sort_values(by='created')

In [4]:
installs=pd.get_dummies(installs, columns=[ "attributed",'implicit','wifi'])

In [5]:
installs['kind']=installs['kind'].fillna('otro')

In [6]:
installs.head()

,created,application_id,ref_type,ref_hash,click_hash,device_countrycode,device_brand,device_model,session_user_agent,user_agent,...,kind,trans_id,ip_address,device_language,attributed_False,attributed_True,implicit_False,implicit_True,wifi_False,wifi_True
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,...,otro,NaN,8409380583076227678,NaN,1,0,1,0,0,0
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,...,otro,NaN,806493034070542041,NaN,1,0,1,0,0,0
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,NaN,6287817205707153877,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,...,af_app_opened,NaN,6319394158544413414,6.977049e+18,1,0,0,1,1,0
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,...,otro,NaN,4206683315615624585,5.221863e+18,1,0,1,0,0,1
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,...,otro,NaN,8335834010411202437,7.528974e+18,1,0,1,0,0,1


In [7]:
installs=installs.drop(['click_hash','device_countrycode'],axis=1)

In [8]:
installs.head()

,created,application_id,ref_type,ref_hash,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,trans_id,ip_address,device_language,attributed_False,attributed_True,implicit_False,implicit_True,wifi_False,wifi_True
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,NaN,Apsalar-Postback,NaN,NaN,otro,NaN,8409380583076227678,NaN,1,0,1,0,0,0
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,NaN,Apsalar-Postback,NaN,NaN,otro,NaN,806493034070542041,NaN,1,0,1,0,0,0
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,NaN,6319394158544413414,6.977049e+18,1,0,0,1,1,0
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,otro,NaN,4206683315615624585,5.221863e+18,1,0,1,0,0,1
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,otro,NaN,8335834010411202437,7.528974e+18,1,0,1,0,0,1


In [9]:
installs=installs.drop(['event_uuid','trans_id'],axis=1)
installs.head()

,created,application_id,ref_type,ref_hash,device_brand,device_model,session_user_agent,user_agent,kind,ip_address,device_language,attributed_False,attributed_True,implicit_False,implicit_True,wifi_False,wifi_True
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,NaN,Apsalar-Postback,NaN,otro,8409380583076227678,NaN,1,0,1,0,0,0
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,NaN,Apsalar-Postback,NaN,otro,806493034070542041,NaN,1,0,1,0,0,0
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,af_app_opened,6319394158544413414,6.977049e+18,1,0,0,1,1,0
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,otro,4206683315615624585,5.221863e+18,1,0,1,0,0,1
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,otro,8335834010411202437,7.528974e+18,1,0,1,0,0,1


In [10]:
#installs['attributed_False']=installs.groupby('ref_hash')['attributed_False'].cumsum()

In [11]:
#installs['attributed_True']=installs.groupby('ref_hash')['attributed_True'].cumsum()

In [12]:
list(installs.columns.values)

['created',
 'application_id',
 'ref_type',
 'ref_hash',
 'device_brand',
 'device_model',
 'session_user_agent',
 'user_agent',
 'kind',
 'ip_address',
 'device_language',
 'attributed_False',
 'attributed_True',
 'implicit_False',
 'implicit_True',
 'wifi_False',
 'wifi_True']

In [13]:
#installs['implicit_False']=installs.groupby('ref_hash')['implicit_False'].cumsum()

In [14]:
#installs['implicit_True']=installs.groupby('ref_hash')['implicit_True'].cumsum()

In [15]:
installs['kind'].value_counts()

otro                            378343
Open                             37257
af_app_opened                    27845
app_open                          8635
app open                          7134
Sign In                           3822
list view                         3665
OPEN                              3271
login_success                     1627
pax_app_open                      1570
product view                       800
EVENT_OPEN_APP                     679
product_preview                    646
account_summary_first_step         505
deeplink_open                      388
Pre_checkout                       365
adjust_uninstall                   338
reattribution                      332
Add_to_cart                        292
af_view_product                    228
sessionbegin                       198
af app open                        154
af_app_opend                       154
1st_ride                           141
af_content_view                    137
MobileSignup             

In [16]:
contador_aplicaciones=installs[['ref_hash','application_id']].drop_duplicates().groupby('ref_hash').cumcount() + 1
contador_aplicaciones=contador_aplicaciones.reindex(installs.index).ffill()

In [17]:
contador_aplicaciones=contador_aplicaciones.to_frame()
contador_aplicaciones.head()

,0
94738,1.0
94737,1.0
111877,1.0
76146,1.0
435184,1.0


In [18]:
contador_aplicaciones=contador_aplicaciones.rename(columns = {0:'aplicaciones_distintas_ingresadas'})

In [19]:
contador_aplicaciones.to_csv('data/installs_contador_aplicaciones.csv')

In [20]:
contador_ip=installs[['ref_hash','ip_address']].drop_duplicates().groupby('ref_hash').cumcount() + 1
contador_ip=contador_ip.reindex(installs.index).ffill()
contador_ip=contador_ip.to_frame()
contador_ip.head()

,0
94738,1.0
94737,1.0
111877,1.0
76146,1.0
435184,1.0


In [21]:
app_mas_visitada=installs.groupby(['ref_hash','application_id']).cumcount() +1
app_mas_visitada=app_mas_visitada.to_frame()

app_mas_visitada=app_mas_visitada.rename(columns = {0:'conteo_en_app_mas_visitada'})
app_mas_visitada.head(30)

,conteo_en_app_mas_visitada
94738,1
94737,1
111877,1
76146,1
435184,1
369451,1
111862,2
111889,1
440883,1
465248,1


In [22]:
installs=installs.merge(app_mas_visitada,how='left', left_index=True, right_index=True)
installs.head()

,created,application_id,ref_type,ref_hash,device_brand,device_model,session_user_agent,user_agent,kind,ip_address,device_language,attributed_False,attributed_True,implicit_False,implicit_True,wifi_False,wifi_True,conteo_en_app_mas_visitada
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,NaN,Apsalar-Postback,NaN,otro,8409380583076227678,NaN,1,0,1,0,0,0,1
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,NaN,Apsalar-Postback,NaN,otro,806493034070542041,NaN,1,0,1,0,0,0,1
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,af_app_opened,6319394158544413414,6.977049e+18,1,0,0,1,1,0,1
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,otro,4206683315615624585,5.221863e+18,1,0,1,0,0,1,1
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,otro,8335834010411202437,7.528974e+18,1,0,1,0,0,1,1


In [23]:
contador_session_user_agent=installs[['ref_hash','session_user_agent']].drop_duplicates().groupby('ref_hash').cumcount() + 1
contador_session_user_agent=contador_session_user_agent.reindex(installs.index).ffill()

In [24]:
contador_session_user_agent=contador_session_user_agent.to_frame()
contador_session_user_agent.head()

,0
94738,1.0
94737,1.0
111877,1.0
76146,1.0
435184,1.0


In [25]:
contador_session_user_agent=contador_session_user_agent.rename(columns = {0:'session_user_agent_distintas_ingresadas'})

In [26]:
contador_session_user_agent.to_csv('data/installs_contador_session_user_agent.csv')

In [27]:
installs=installs.merge(contador_session_user_agent,how='left', left_index=True, right_index=True)

In [28]:
contador_ip=installs[['ref_hash','ip_address']].drop_duplicates().groupby('ref_hash').cumcount() + 1
contador_ip=contador_ip.reindex(installs.index).ffill()
contador_ip=contador_ip.to_frame()
contador_ip.head()
contador_ip=contador_ip.rename({0:'ip_distintas_visitadas'})

In [29]:
contador_app_mas_instalada=installs.groupby(['ref_hash','application_id']).cumcount() +1
contador_app_mas_instalada=contador_app_mas_instalada.to_frame()

contador_app_mas_instalada=contador_app_mas_instalada.rename(columns = {0:'application_id_mas_instalada'})
contador_app_mas_instalada.head()

,application_id_mas_instalada
94738,1
94737,1
111877,1
76146,1
435184,1


In [30]:
installs=installs.merge(contador_app_mas_instalada,how='left', left_index=True, right_index=True)
installs.head()


,created,application_id,ref_type,ref_hash,device_brand,device_model,session_user_agent,user_agent,kind,ip_address,device_language,attributed_False,attributed_True,implicit_False,implicit_True,wifi_False,wifi_True,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,application_id_mas_instalada
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,NaN,Apsalar-Postback,NaN,otro,8409380583076227678,NaN,1,0,1,0,0,0,1,1.0,1
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,NaN,Apsalar-Postback,NaN,otro,806493034070542041,NaN,1,0,1,0,0,0,1,1.0,1
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,af_app_opened,6319394158544413414,6.977049e+18,1,0,0,1,1,0,1,1.0,1
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,otro,4206683315615624585,5.221863e+18,1,0,1,0,0,1,1,1.0,1
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,otro,8335834010411202437,7.528974e+18,1,0,1,0,0,1,1,1.0,1


In [31]:
installs['application_id']=installs['application_id'].apply(hash)# si no se hashea no sirve para predecir
app_mas_instalada=installs[['ref_hash','application_id','application_id_mas_instalada']]
app_mas_instalada=app_mas_instalada.groupby('ref_hash')['application_id'].agg({'application_id_mas_instalada' : "cummax"})
app_mas_instalada=app_mas_instalada.rename({'application_id_mas_instalada':'app_mas_instalada'},axis=1)
app_mas_instalada.head()

D:\Users\Lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,app_mas_instalada
94738,70
94737,70
111877,65
76146,27
435184,339


In [32]:
installs=installs.merge(app_mas_instalada,how='left', left_index=True, right_index=True)

In [33]:
installs.head()

,created,application_id,ref_type,ref_hash,device_brand,device_model,session_user_agent,user_agent,kind,ip_address,...,attributed_False,attributed_True,implicit_False,implicit_True,wifi_False,wifi_True,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,application_id_mas_instalada,app_mas_instalada
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,NaN,Apsalar-Postback,NaN,otro,8409380583076227678,...,1,0,1,0,0,0,1,1.0,1,70
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,NaN,Apsalar-Postback,NaN,otro,806493034070542041,...,1,0,1,0,0,0,1,1.0,1,70
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,af_app_opened,6319394158544413414,...,1,0,0,1,1,0,1,1.0,1,65
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,otro,4206683315615624585,...,1,0,1,0,0,1,1,1.0,1,27
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,otro,8335834010411202437,...,1,0,1,0,0,1,1,1.0,1,339


In [34]:
installs['session_user_agent']=installs['session_user_agent'].apply(hash)
installs_por_session_user_agent_mayor=installs[['ref_hash','session_user_agent','session_user_agent_distintas_ingresadas']]
installs_por_session_user_agent_mayor=installs_por_session_user_agent_mayor.groupby('ref_hash')['session_user_agent'].agg({'session_user_agent_distintas_ingresadas' : "cummax"})
installs_por_session_user_agent_mayor=installs_por_session_user_agent_mayor.rename({'session_user_agent_distintas_ingresadas':'session_user_agent_mas_ingresada'},axis=1)
installs_por_session_user_agent_mayor.head()


#SEGUIR ESTO

D:\Users\Lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,session_user_agent_mas_ingresada
94738,-8238431572677170032
94737,-8238431572677170032
111877,-67021364715795598
76146,-67021364715795598
435184,-67021364715795598


In [35]:
installs=installs.merge(installs_por_session_user_agent_mayor,how='left', left_index=True, right_index=True)
installs.head()

,created,application_id,ref_type,ref_hash,device_brand,device_model,session_user_agent,user_agent,kind,ip_address,...,attributed_True,implicit_False,implicit_True,wifi_False,wifi_True,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,application_id_mas_instalada,app_mas_instalada,session_user_agent_mas_ingresada
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,NaN,-8238431572677170032,NaN,otro,8409380583076227678,...,0,1,0,0,0,1,1.0,1,70,-8238431572677170032
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,NaN,-8238431572677170032,NaN,otro,806493034070542041,...,0,1,0,0,0,1,1.0,1,70,-8238431572677170032
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,3.083059e+17,5.274185e+18,-67021364715795598,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,af_app_opened,6319394158544413414,...,0,0,1,1,0,1,1.0,1,65,-67021364715795598
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6.794880e+18,-67021364715795598,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,otro,4206683315615624585,...,0,1,0,0,1,1,1.0,1,27,-67021364715795598
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6.794880e+18,-67021364715795598,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,otro,8335834010411202437,...,0,1,0,0,1,1,1.0,1,339,-67021364715795598


In [36]:
installs['kind'].value_counts()

otro                            378343
Open                             37257
af_app_opened                    27845
app_open                          8635
app open                          7134
Sign In                           3822
list view                         3665
OPEN                              3271
login_success                     1627
pax_app_open                      1570
product view                       800
EVENT_OPEN_APP                     679
product_preview                    646
account_summary_first_step         505
deeplink_open                      388
Pre_checkout                       365
adjust_uninstall                   338
reattribution                      332
Add_to_cart                        292
af_view_product                    228
sessionbegin                       198
af app open                        154
af_app_opend                       154
1st_ride                           141
af_content_view                    137
MobileSignup             

In [37]:
kind_mas_visitada=installs.groupby(['ref_hash','kind']).cumcount() +1
kind_mas_visitada=kind_mas_visitada.to_frame()

kind_mas_visitada=kind_mas_visitada.rename(columns = {0:'conteo_en_kind_mas_visitada'})
kind_mas_visitada.head(30)

,conteo_en_kind_mas_visitada
94738,1
94737,1
111877,1
76146,1
435184,1
369451,1
111862,1
111889,1
440883,1
465248,1


In [38]:
installs=installs.merge(kind_mas_visitada,how='left', left_index=True, right_index=True)

In [39]:
installs.dtypes

created                                    datetime64[ns]
application_id                                      int64
ref_type                                            int64
ref_hash                                         category
device_brand                                      float64
device_model                                      float64
session_user_agent                                  int64
user_agent                                         object
kind                                               object
ip_address                                          int64
device_language                                   float64
attributed_False                                    uint8
attributed_True                                     uint8
implicit_False                                      uint8
implicit_True                                       uint8
wifi_False                                          uint8
wifi_True                                           uint8
conteo_en_app_

In [40]:
installs['kind']=installs['kind'].apply(hash)# si no se hashea no sirve para predecir
installs_por_kind_mas_importante=installs[['ref_hash','kind','conteo_en_kind_mas_visitada']]
installs_por_kind_mas_importante=installs_por_kind_mas_importante.groupby('ref_hash')['kind'].agg({'conteo_en_kind_mas_visitada' : "cummax"})
installs_por_kind_mas_importante=installs_por_kind_mas_importante.rename({'conteo_en_kind_mas_visitada':'kind_mas_instalada'},axis=1)
installs_por_kind_mas_importante.head()

D:\Users\Lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,kind_mas_instalada
94738,3198436769233378858
94737,3198436769233378858
111877,-4147052740965887152
76146,3198436769233378858
435184,3198436769233378858


In [41]:
installs=installs.merge(installs_por_kind_mas_importante,how='left', left_index=True, right_index=True)

In [42]:
installs.head()

,created,application_id,ref_type,ref_hash,device_brand,device_model,session_user_agent,user_agent,kind,ip_address,...,implicit_True,wifi_False,wifi_True,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,application_id_mas_instalada,app_mas_instalada,session_user_agent_mas_ingresada,conteo_en_kind_mas_visitada,kind_mas_instalada
94738,2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,NaN,-8238431572677170032,NaN,3198436769233378858,8409380583076227678,...,0,0,0,1,1.0,1,70,-8238431572677170032,1,3198436769233378858
94737,2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,NaN,-8238431572677170032,NaN,3198436769233378858,806493034070542041,...,0,0,0,1,1.0,1,70,-8238431572677170032,1,3198436769233378858
111877,2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,3.083059e+17,5.274185e+18,-67021364715795598,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,-4147052740965887152,6319394158544413414,...,1,1,0,1,1.0,1,65,-67021364715795598,1,-4147052740965887152
76146,2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,6.794880e+18,-67021364715795598,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,3198436769233378858,4206683315615624585,...,0,0,1,1,1.0,1,27,-67021364715795598,1,3198436769233378858
435184,2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,6.794880e+18,-67021364715795598,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,3198436769233378858,8335834010411202437,...,0,0,1,1,1.0,1,339,-67021364715795598,1,3198436769233378858


In [43]:
#fentana para features de train
fecha_minima=pd.to_datetime('2019-04-18 00:00:00')
fecha_tope=pd.to_datetime('2019-04-21 00:00:00')

installs_ventana1=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de segundos
installs_ventana1['tiempo_hasta_aparicion']=(installs_ventana1['created']-fecha_minima)/ np.timedelta64(1, 's')
installs_ventana1_reducido=installs_ventana1.groupby(['ref_hash']).aggregate({"created" : min,\
                                                                         'attributed_False': 'sum',\
                                                                        'attributed_True': 'sum',\
                                                                        'conteo_en_kind_mas_visitada':'sum',\
                                                                        'app_mas_instalada':'sum',\
                                                                        'conteo_en_app_mas_visitada':'sum',\
                                                                        'session_user_agent_distintas_ingresadas':'sum',\
                                                                        #'session_user_agent_mas_ingresada ':'last',\
                                                                        'wifi_True':'sum',\
                                                                        'wifi_False':'sum',\
                                                                         'implicit_True':'sum',\
                                                                        'implicit_False': 'sum',\
                                                                           'tiempo_hasta_aparicion':"min"})\
                                                                            .reset_index('ref_hash')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
installs_ventana1_reducido=installs_ventana1_reducido.dropna()
#installs_ventana3_reducido['ref_hash'].value_counts()

In [44]:

features_ventana1=installs_ventana1_reducido.drop(['created'],axis=1)
features_ventana1.head()

,ref_hash,attributed_False,attributed_True,conteo_en_kind_mas_visitada,app_mas_instalada,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,wifi_True,wifi_False,implicit_True,implicit_False,tiempo_hasta_aparicion
0,1000001243847627208,1,0,1,309,1,1.0,0,0,0,1,179017.206
6,1000138003886484266,1,0,1,36,1,1.0,1,0,0,1,165429.663
8,1000172260833750114,1,0,1,167,1,1.0,0,0,0,1,72391.763
10,1000214925038058238,1,0,1,36,1,1.0,1,0,0,1,185678.575
12,1000280583608032134,1,0,1,54,1,1.0,0,0,0,1,257675.026


In [45]:
#labels para predecir, features para test

fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')

installs_ventana3=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de segundos
installs_ventana3['tiempo_hasta_aparicion']=(installs_ventana3['created']-fecha_minima)/ np.timedelta64(1, 's')
installs_ventana3_reducido=installs_ventana3.groupby(['ref_hash']).aggregate({"created" : min,\
                                                                         'attributed_False': 'sum',\
                                                                        'attributed_True': 'sum',\
                                                                        'conteo_en_kind_mas_visitada':'sum',\
                                                                        'app_mas_instalada':'sum',\
                                                                        'conteo_en_app_mas_visitada':'sum',\
                                                                        'session_user_agent_distintas_ingresadas':'sum',\
                                                                        #'session_user_agent_mas_ingresada ':'first',\
                                                                        'wifi_True':'sum',\
                                                                        'wifi_False':'sum',\
                                                                         'implicit_True':'sum',\
                                                                        'implicit_False': 'sum',\
                                                                           'tiempo_hasta_aparicion':"min"})\
                                                                            .reset_index('ref_hash')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
installs_ventana3_reducido=installs_ventana3_reducido.dropna()


In [46]:
features_ventana3=installs_ventana3_reducido.drop('created',axis=1)
features_ventana3.head()

,ref_hash,attributed_False,attributed_True,conteo_en_kind_mas_visitada,app_mas_instalada,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,wifi_True,wifi_False,implicit_True,implicit_False,tiempo_hasta_aparicion
2,1000080701377762047,1,0,1,36,1,1.0,1,0,0,1,186069.591
4,1000095322020146100,1,0,1,121,1,1.0,1,0,0,1,94037.085
5,100010331276132711,1,0,1,210,1,1.0,0,0,0,1,149899.132
7,1000169251625791246,1,0,1,122,1,1.0,0,0,0,1,29646.178
13,1000288182047842384,1,0,1,210,1,1.0,0,0,0,1,243306.923


In [47]:
labels_ventana3=installs_ventana3_reducido[['ref_hash','tiempo_hasta_aparicion']].copy()
labels_ventana3.head()

,ref_hash,tiempo_hasta_aparicion
2,1000080701377762047,186069.591
4,1000095322020146100,94037.085
5,100010331276132711,149899.132
7,1000169251625791246,29646.178
13,1000288182047842384,243306.923


In [48]:
features_ventana1=installs_ventana1_reducido.drop('created',axis=1)
labels_ventana3.head()
features_ventana1=features_ventana1.rename(columns = {'tiempo_hasta_aparicion':'tiempo_hasta_aparicion_viejo'})
set_train=labels_ventana3.merge(features_ventana1,how='left', left_on='ref_hash',right_on='ref_hash')
#set_train['tiempo_hasta_aparicion_viejo']=set_train['tiempo_hasta_aparicion_viejo'].fillna(-1)
#set_train=set_train.fillna(value=0)
set_train[['attributed_False','attributed_True','conteo_en_kind_mas_visitada','conteo_en_app_mas_visitada','session_user_agent_distintas_ingresadas','wifi_True','wifi_False','implicit_True','implicit_False']]=set_train[['attributed_False','attributed_True','conteo_en_kind_mas_visitada','conteo_en_app_mas_visitada','session_user_agent_distintas_ingresadas','wifi_True','wifi_False','implicit_True','implicit_False']].fillna(0)

In [49]:
set_train.head()

,ref_hash,tiempo_hasta_aparicion,attributed_False,attributed_True,conteo_en_kind_mas_visitada,app_mas_instalada,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,wifi_True,wifi_False,implicit_True,implicit_False,tiempo_hasta_aparicion_viejo
0,1000080701377762047,186069.591,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1000095322020146100,94037.085,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,100010331276132711,149899.132,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1000169251625791246,29646.178,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1000288182047842384,243306.923,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [50]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')

installs_ventana5=installs.loc[((installs['created']>fecha_minima) & (installs['created']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de segundos
installs_ventana5['tiempo_hasta_aparicion']=(installs_ventana5['created']-fecha_minima)/ np.timedelta64(1, 's')
installs_ventana5=installs_ventana5.groupby(['ref_hash']).aggregate({"created" : min,\
                                                                         'attributed_False': 'first',\
                                                                        'attributed_True': 'first',\
                                                                        'conteo_en_kind_mas_visitada':'first',\
                                                                        'app_mas_instalada':'first',\
                                                                        'conteo_en_app_mas_visitada':'first',\
                                                                        'session_user_agent_distintas_ingresadas':'first',\
                                                                        #'session_user_agent_mas_ingresada ':'first',\
                                                                        'wifi_True':'first',\
                                                                        'wifi_False':'first',\
                                                                         'implicit_True':'first',\
                                                                        'implicit_False': 'first',\
                                                                        #'entradas_previas':min,\
                                                                           'tiempo_hasta_aparicion':"min"})\
                                                                            .reset_index('ref_hash')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
installs_ventana5=installs_ventana5.dropna()

In [51]:
features_ventana5=installs_ventana3_reducido.drop('created',axis=1)

labels_ventana5=installs_ventana5[['ref_hash','tiempo_hasta_aparicion']].copy()

features_ventana5=features_ventana5.rename(columns = {'tiempo_hasta_aparicion':'tiempo_hasta_aparicion_viejo'})

set_test=labels_ventana5.merge(features_ventana1,how='left', left_on='ref_hash',right_on='ref_hash')
#set_test['tiempo_hasta_aparicion_viejo']=set_test['tiempo_hasta_aparicion_viejo'].fillna(-1)
#set_train=set_train.fillna(value=0)
set_test[['attributed_False','attributed_True','conteo_en_kind_mas_visitada','conteo_en_app_mas_visitada','session_user_agent_distintas_ingresadas','wifi_True','wifi_False','implicit_True','implicit_False']]=set_test[['attributed_False','attributed_True','conteo_en_kind_mas_visitada','conteo_en_app_mas_visitada','session_user_agent_distintas_ingresadas','wifi_True','wifi_False','implicit_True','implicit_False']].fillna(0)
set_test.head()

,ref_hash,tiempo_hasta_aparicion,attributed_False,attributed_True,conteo_en_kind_mas_visitada,app_mas_instalada,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,wifi_True,wifi_False,implicit_True,implicit_False,tiempo_hasta_aparicion_viejo
0,1000061425870948777,181422.436,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1000085014918096773,5888.649,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1000193663803871158,238907.595,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1000252911429490816,241592.080,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1000298337185304784,188550.814,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [52]:
installs_ventana5.head()

,ref_hash,created,attributed_False,attributed_True,conteo_en_kind_mas_visitada,app_mas_instalada,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,wifi_True,wifi_False,implicit_True,implicit_False,tiempo_hasta_aparicion
1,1000061425870948777,2019-04-26 02:23:42.436,1.0,0.0,1.0,122.0,1.0,1.0,0.0,0.0,0.0,1.0,181422.436
3,1000085014918096773,2019-04-24 01:38:08.649,1.0,0.0,1.0,86.0,1.0,1.0,0.0,0.0,0.0,1.0,5888.649
9,1000193663803871158,2019-04-26 18:21:47.595,1.0,0.0,1.0,36.0,1.0,1.0,1.0,0.0,0.0,1.0,238907.595
11,1000252911429490816,2019-04-26 19:06:32.080,1.0,0.0,1.0,121.0,1.0,1.0,1.0,0.0,0.0,1.0,241592.080
16,1000298337185304784,2019-04-26 04:22:30.814,1.0,0.0,1.0,14.0,1.0,1.0,0.0,0.0,0.0,1.0,188550.814


In [53]:
competencia = pd.read_csv("data/target_competencia_ids.csv", dtype={'ref_hash': 'category'})
competencia[['ref_hash','prediccion']] = competencia['ref_hash'].str.split('_',expand=True)
competencia.head()
competencia=competencia.loc[competencia['prediccion']=='st']
competencia=competencia.drop(['obj','prediccion'],axis=1)
competencia.head()

set_test=set_test.merge(competencia,how='right',left_on='ref_hash',right_on='ref_hash')

In [54]:
#set_test['tiempo_hasta_aparicion_viejo']=set_test['tiempo_hasta_aparicion_viejo'].fillna(259200)
set_test['tiempo_hasta_aparicion']=set_test['tiempo_hasta_aparicion'].fillna(259200)
set_test.head()

,ref_hash,tiempo_hasta_aparicion,attributed_False,attributed_True,conteo_en_kind_mas_visitada,app_mas_instalada,conteo_en_app_mas_visitada,session_user_agent_distintas_ingresadas,wifi_True,wifi_False,implicit_True,implicit_False,tiempo_hasta_aparicion_viejo
0,1010070503877148763,153560.756,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1013543838965040946,164925.205,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,101763096421653092,178348.928,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1017958004932387739,10937.657,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1018126962792619181,247358.507,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [55]:
list(set_test.columns.values)

['ref_hash',
 'tiempo_hasta_aparicion',
 'attributed_False',
 'attributed_True',
 'conteo_en_kind_mas_visitada',
 'app_mas_instalada',
 'conteo_en_app_mas_visitada',
 'session_user_agent_distintas_ingresadas',
 'wifi_True',
 'wifi_False',
 'implicit_True',
 'implicit_False',
 'tiempo_hasta_aparicion_viejo']

In [56]:
columnsTitles = list(set_train.columns.values)
installs_ventana3 = set_train.reindex(columns=columnsTitles)
#installs_ventana3 = installs_ventana3.set_index('ref_hash')
train_features = installs_ventana3.drop(['tiempo_hasta_aparicion'],axis=1)
train_features.reset_index().to_csv('xgb/train_features_prediccion_installs.csv')
train_labels = set_train.drop(['session_user_agent_distintas_ingresadas','attributed_False','attributed_True','conteo_en_kind_mas_visitada','app_mas_instalada'\
                                       ,'conteo_en_app_mas_visitada','wifi_True','wifi_False','implicit_True','implicit_False','tiempo_hasta_aparicion_viejo'],axis=1)
train_labels.reset_index().to_csv('xgb/train_labels.csv')
print(train_features.head())
print(train_labels.head())
installs_ventana3_reducido.dtypes
train_features=train_features.set_index('ref_hash')
train_labels=train_labels.set_index('ref_hash')

              ref_hash  attributed_False  attributed_True  \
0  1000080701377762047               0.0              0.0   
1  1000095322020146100               0.0              0.0   
2   100010331276132711               0.0              0.0   
3  1000169251625791246               0.0              0.0   
4  1000288182047842384               0.0              0.0   

   conteo_en_kind_mas_visitada  app_mas_instalada  conteo_en_app_mas_visitada  \
0                          0.0                NaN                         0.0   
1                          0.0                NaN                         0.0   
2                          0.0                NaN                         0.0   
3                          0.0                NaN                         0.0   
4                          0.0                NaN                         0.0   

   session_user_agent_distintas_ingresadas  wifi_True  wifi_False  \
0                                      0.0        0.0         0.0   
1       

In [57]:
columnsTitles = list(set_test.columns.values)
installs_ventana5_reducido = set_test.reindex(columns=columnsTitles)
test_features = set_test.drop(['tiempo_hasta_aparicion'],axis=1)
test_features.reset_index().to_csv('xgb/test_features_prediccion_auctions.csv')
test_labels = set_test.drop(['session_user_agent_distintas_ingresadas','attributed_False','attributed_True','conteo_en_kind_mas_visitada','app_mas_instalada'\
                                       ,'conteo_en_app_mas_visitada','wifi_True','wifi_False','implicit_True','implicit_False','tiempo_hasta_aparicion_viejo'],axis=1)
test_labels.reset_index().to_csv('xgb/test_labels.csv')
print(test_features.head())
print(test_labels.head())
test_features=test_features.set_index('ref_hash')
test_labels=test_labels.set_index('ref_hash')

              ref_hash  attributed_False  attributed_True  \
0  1010070503877148763               0.0              0.0   
1  1013543838965040946               0.0              0.0   
2   101763096421653092               0.0              0.0   
3  1017958004932387739               0.0              0.0   
4  1018126962792619181               0.0              0.0   

   conteo_en_kind_mas_visitada  app_mas_instalada  conteo_en_app_mas_visitada  \
0                          0.0                NaN                         0.0   
1                          0.0                NaN                         0.0   
2                          0.0                NaN                         0.0   
3                          0.0                NaN                         0.0   
4                          0.0                NaN                         0.0   

   session_user_agent_distintas_ingresadas  wifi_True  wifi_False  \
0                                      0.0        0.0         0.0   
1       

In [90]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.01, learning_rate = 0.1,
                max_depth = 1, alpha = 150, n_estimators = 550,
                         scale_pos_weight=15,n_jobs=600,subsample=0.1,nfolds=10)

            #mas n_estimators es mejor, pero tarda mas
            #subsample mejora algo

In [91]:
xg_reg.fit(train_features,train_labels)

XGBRegressor(alpha=150, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.01, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=1, min_child_weight=1, missing=None, n_estimators=550,
       n_jobs=600, nfolds=10, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=15,
       seed=None, silent=True, subsample=0.1)

In [92]:
predicted_labels = xg_reg.predict(test_features)

In [93]:
rmse = np.sqrt(mean_squared_error(test_labels, predicted_labels))
print("RMSE: %f" % (rmse))

RMSE: 83891.117798
